# PyCaret 2 Regression Example
- https://github.com/pycaret/pycaret/blob/master/examples/PyCaret%202%20Regression.ipynb

In [1]:
from pycaret.utils import version
version()

'2.2.2'

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))# デフォルトは75%

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm_notebook as tqdm
import pycaret

sns.set()

# 常に全ての列（カラム）を表示
pd.set_option("display.max_columns", None)

!pwd
sys.executable

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\atmaCup_#8\notebook\pycaret2\20201208


'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [3]:
# パラメ
# https://pycaret.readthedocs.io/en/latest/api/regression.html

# 出力ディレクトリ
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# 目的変数
target = "Global_Sales"

# 乱数シード
session_id = 123

# cv hold
fold = 5
fold_strategy = "groupkfold"
fold_groups = "Publisher"

# metric
optimize = "RMSLE"

# models
choice_ms = ["et", "catboost", "lightgbm", "gbr", "huber"]

# チューニング回数
n_iter = 100

#DEBUG = True
DEBUG = False
if DEBUG:
    n_iter = 2

params = dict(
    target=target,
    session_id=session_id,
    fold=fold,
    fold_strategy=fold_strategy,
    fold_groups=fold_groups,
    fold_shuffle=True,
    silent=True,
    use_gpu=True,
)

In [4]:
import pandas as pd

#DATADIR = r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\atmaCup_#8\data\atmacup08-dataset"
#train = pd.read_csv(f"{DATADIR}/train.csv")
#test = pd.read_csv(f"{DATADIR}/test.csv")

#DATADIR = r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\atmaCup_#8\notebook\data_check"
#train = pd.read_csv(f"{DATADIR}/train_preprocess.csv")
#test = pd.read_csv(f"{DATADIR}/test_preprocess.csv")

DATADIR = r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\atmaCup_#8\notebook\lgbm\20201207_Publisher_GroupkFold_seed"
train = pd.read_csv(f"{DATADIR}/train_fe.csv")
test = pd.read_csv(f"{DATADIR}/test_fe.csv")

train = train.loc[:, test.columns.to_list() + ["Global_Sales"]]
train

,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,User_Score_was_missing,Critic_Score_was_missing,Critic_Count_was_missing,User_Count_was_missing,series,log_User_Count,row_Critic_Score_User_Score_sum,row_Critic_Score_User_Score_mean,row_Critic_Score_User_Score_std,row_Critic_Count_User_Count_sum,row_Critic_Count_User_Count_mean,row_Critic_Count_User_Count_std,Platform_User_Count_max,Platform_User_Count_min,Platform_User_Count_mean,Platform_User_Count_std,Platform_User_Count_skew,Platform_User_Count_kurt,Platform_Critic_Count_max,Platform_Critic_Count_min,Platform_Critic_Count_mean,Platform_Critic_Count_std,Platform_Critic_Count_skew,Platform_Critic_Count_kurt,Platform_User_Score_max,Platform_User_Score_min,Platform_User_Score_mean,Platform_User_Score_std,Platform_User_Score_skew,Platform_User_Score_kurt,Platform_Critic_Score_max,Platform_Critic_Score_min,Platform_Critic_Score_mean,Platform_Critic_Score_std,Platform_Critic_Score_skew,Platform_Critic_Score_kurt,Genre_User_Count_max,Genre_User_Count_min,Genre_User_Count_mean,Genre_User_Count_std,Genre_User_Count_skew,Genre_User_Count_kurt,Genre_Critic_Count_max,Genre_Critic_Count_min,Genre_Critic_Count_mean,Genre_Critic_Count_std,Genre_Critic_Count_skew,Genre_Critic_Count_kurt,Genre_User_Score_max,Genre_User_Score_min,Genre_User_Score_mean,Genre_User_Score_std,Genre_User_Score_skew,Genre_User_Score_kurt,Genre_Critic_Score_max,Genre_Critic_Score_min,Genre_Critic_Score_mean,Genre_Critic_Score_std,Genre_Critic_Score_skew,Genre_Critic_Score_kurt,Publisher_User_Count_max,Publisher_User_Count_min,Publisher_User_Count_mean,Publisher_User_Count_std,Publisher_User_Count_skew,Publisher_User_Count_kurt,Publisher_Critic_Count_max,Publisher_Critic_Count_min,Publisher_Critic_Count_mean,Publisher_Critic_Count_std,Publisher_Critic_Count_skew,Publisher_Critic_Count_kurt,Publisher_User_Score_max,Publisher_User_Score_min,Publisher_User_Score_mean,Publisher_User_Score_std,Publisher_User_Score_skew,Publisher_User_Score_kurt,Publisher_Critic_Score_max,Publisher_Critic_Score_min,Publisher_Critic_Score_mean,Publisher_Critic_Score_std,Publisher_Critic_Score_skew,Publisher_Critic_Score_kurt,Developer_User_Count_max,Developer_User_Count_min,Developer_User_Count_mean,Developer_User_Count_std,Developer_User_Count_skew,Developer_User_Count_kurt,Developer_Critic_Count_max,Developer_Critic_Count_min,Developer_Critic_Count_mean,Developer_Critic_Count_std,Developer_Critic_Count_skew,Developer_Critic_Count_kurt,Developer_User_Score_max,Developer_User_Score_min,Developer_User_Score_mean,Developer_User_Score_std,Developer_User_Score_skew,Developer_User_Score_kurt,Developer_Critic_Score_max,Developer_Critic_Score_min,Developer_Critic_Score_mean,Developer_Critic_Score_std,Developer_Critic_Score_skew,Developer_Critic_Score_kurt,Rating_User_Count_max,Rating_User_Count_min,Rating_User_Count_mean,Rating_User_Count_std,Rating_User_Count_skew,Rating_User_Count_kurt,Rating_Critic_Count_max,Rating_Critic_Count_min,Rating_Critic_Count_mean,Rating_Critic_Count_std,Rating_Critic_Count_skew,Rating_Critic_Count_kurt,Rating_User_Score_max,Rating_User_Score_min,Rating_User_Score_mean,Rating_User_Score_std,Rating_User_Score_skew,Rating_User_Score_kurt,Rating_Critic_Score_max,Rating_Critic_Score_min,Rating_Critic_Score_mean,Rating_Critic_Score_std,Rating_Critic_Score_skew,Rating_Critic_Score_kurt,Critic_Score_*_Critic_Count,User_Score_*_User_Count,Critic_Score_*_User_Score,Critic_Count_*_User_Count,Critic_Count_+_User_Count,Critic_Count_-_User_Count,Critic_Count_/_all_Count,clusters_cate_cols,clusters_Score_Count,Global_Sales
0,0,0,0,0,0,74.000000,17.000000,7.9,22.000000,0,0,0,0,0,0,0,3.135494,81.900000,40.950000,46.739758,39.000000,19.500000,3.535534,2147.000000,4.000000,115.942220,134.680370,8.680930,112.286301,87.000000,4.000000,24.094851,11.556586,1.281734,5.078568,9.3,0.2,7.351439,1.247140,-2.681698,8.297436,97.000000,19.000000,66.244882,10.359427,-1.509346,3.764847,80

In [5]:
from pycaret.regression import *
reg1 = setup(train, **params)

,Description,Value
0,session_id,123
1,Target,Global_Sales
2,Original Data,"(8359, 153)"
3,Missing Values,True
4,Numeric Features,144
5,Categorical Features,8
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(5851, 141)"


In [6]:
best_model = compare_models(fold=fold,
                            groups=fold_groups,
                            sort=optimize)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,52.7486,25585.2286,128.9924,0.1983,1.2440,3.1136,3.3680
huber,Huber Regressor,52.0482,30857.0566,141.6627,0.0410,1.2572,1.9588,0.6500
gbr,Gradient Boosting Regressor,55.1764,26257.0412,132.5097,0.1216,1.2746,3.4393,3.1840
lightgbm,Light Gradient Boosting Machine,55.7567,26537.6375,132.5794,0.1200,1.2907,3.4152,0.1160
et,Extra Trees Regressor,53.5701,24936.4668,125.5470,0.2512,1.3056,3.7501,1.6500
xgboost,Extreme Gradient Boosting,58.9967,28859.7322,143.5484,-0.1358,1.3404,3.8262,1.2060
knn,K Neighbors Regressor,61.3602,30990.0323,152.3288,-0.4298,1.3423,3.5875,0.4460
rf,Random Forest Regressor,60.6127,28377.2965,139.1777,-0.0054,1.3547,4.3855,2.3420
dt,Decision Tree Regressor,73.7109,43122.3517,182.9543,-1.0389,1.5133,4.4309,0.2240
br,Bayesian Ridge,70.4505,29404.7610,145.3696,-0.1754,1.5961,6.0504,0.2240


In [7]:
tune_models = []
for name in choice_ms:
    print("model:", name)
    model = create_model(name, 
                         fold=fold, 
                         groups=fold_groups)
    tuned_m = tune_model(
        model,
        fold=fold,
        groups=fold_groups,
        optimize=optimize,
        n_iter=n_iter,
        early_stopping=True,
        early_stopping_max_iters=n_iter//3,
    )
    tune_models.append(tuned_m)
    save_model(tuned_m, model_name=os.path.join(output_dir, "pycaret_tuned_" + name))
print(tune_models)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,31.8383,4264.8346,65.3057,0.0216,1.1469,1.6076
1,118.8267,117433.9955,342.6864,-0.0776,1.5599,1.3155
2,26.3701,3531.5891,59.4272,0.2212,1.1721,2.3827
3,42.3105,12922.0085,113.6750,-0.1281,1.2110,2.4549
4,40.7323,16271.1875,127.5586,0.1602,1.1552,1.9052
Mean,52.0156,30884.7230,141.7306,0.0395,1.2490,1.9332
SD,33.9118,43551.8162,103.9094,0.1340,0.1570,0.4388


Transformation Pipeline and Model Succesfully Saved
[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                    max_depth=9, max_features=1.0, max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.01,
                    min_impurity_split=None, min_samples_leaf=3,
                    min_samples_split=7, min_weight_fraction_leaf=0.0,
                    n_estimators=280, n_jobs=-1, oob_score=False,
                    random_state=123, verbose=0, warm_start=False), <catboost.core.CatBoostRegressor object at 0x000002392A1A3A00>, LGBMRegressor(bagging_fraction=0.8, bagging_freq=2, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.4,
              importance_type='split', learning_rate=0.029, max_depth=-1,
              min_child_samples=10, min_child_weight=0.001, min_split_gain=0.3,
              n_estimators=280, n_jobs=-1, num_leaves=150, objective=None,
              random_state

In [8]:
# groupkfold ではなぜかエラー
#stacker = stack_models(estimator_list=tune_models[:-1], 
#                       meta_model=tune_models[-1],
#                       fold=fold,
#                       groups=fold_groups,
#                       optimize=optimize,
#                      )
#save_model(stacker, model_name=os.path.join(output_dir, "pycaret_stacker"))

In [9]:
evaluate_model(tune_models[0])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [12]:
f_tune_models = []
for name in choice_ms:
    loaded_model = load_model(os.path.join(output_dir, f"pycaret_tuned_{name}"))
    f_tune_models.append(loaded_model)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [13]:
from sklearn.metrics import mean_squared_log_error

def score(y, y_pred):
    RMSLE = np.sqrt(np.mean(((np.log(y + 1) - np.log(y_pred + 1)) ** 2)))
    return RMSLE

def make_submit(df_test, model, output_dir: str, csv_name: str):
    """submit csv作成"""
    df_sub = predict_model(model, data=df_test)[["Label"]].reset_index()
    df_sub.columns = ["id", target]
    df_sub = df_sub[[target]]
    
    df_sub[target] = np.where(df_sub[target]<0, 0, df_sub[target])
    
    df_sub.to_csv(f"{output_dir}/{csv_name}.csv", index=False)
    return df_sub


for m, name in zip(f_tune_models, choice_ms):
    print(f"name: {name}")
    df_sub = make_submit(train.loc[:, test.columns.to_list()], m, output_dir, f"{name}_train_rmsle")
    y = train[target].values
    y_pred = df_sub[target].values
    y_pred = np.where(y_pred<0, 0, y_pred)
    print("train_rmsle:", score(y, y_pred))

for m, name in zip(f_tune_models, choice_ms):
    df_sub = make_submit(test, m, output_dir, f"{name}_submission")
    display(df_sub.head())

name: et
train_rmsle: 0.9050330125883147
name: catboost
train_rmsle: 0.9812237801733604
name: lightgbm
train_rmsle: 0.8506446355854378
name: gbr
train_rmsle: 0.9199061769487875
name: huber
train_rmsle: 1.2353981121251805


,Global_Sales
0,29.085714
1,63.771429
2,57.630357
3,61.808929
4,48.394643


,Global_Sales
0,87.153244
1,74.050052
2,59.280822
3,51.857822
4,45.600481


,Global_Sales
0,273.968344
1,84.430642
2,366.491578
3,20.414253
4,12.388204


,Global_Sales
0,393.777929
1,179.557425
2,622.757724
3,104.169832
4,79.996846


,Global_Sales
0,11.858811
1,16.900072
2,28.296668
3,47.746229
4,34.959025
